In [0]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
import argparse
import re
from keras.models import load_model
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model, Sequential
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Activation
from keras.layers.merge import add
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping, LambdaCallback


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
glove = open('/content/drive/My Drive/Datasets/glove.6B.50d.txt',encoding='utf-8')
d=open('/content/drive/My Drive/Datasets/data.txt').read()


In [0]:
#from google.colab import files
#files.upload()

In [0]:
#d=open('data.txt','r').read()
len(d)

3196213

In [0]:
d=d.lower().replace('\n',' \n ')

In [0]:
d[:100]


'"well, prince, so genoa and lucca are now just family estates of the \n buonapartes. but i warn you, '

In [0]:
text_in_words = [w for w in d.split(' ') if w.strip() != '' or w == '\n']

In [0]:
len(text_in_words)

623708

In [0]:
text_in_words[:100]

['"well,',
 'prince,',
 'so',
 'genoa',
 'and',
 'lucca',
 'are',
 'now',
 'just',
 'family',
 'estates',
 'of',
 'the',
 '\n',
 'buonapartes.',
 'but',
 'i',
 'warn',
 'you,',
 'if',
 'you',
 "don't",
 'tell',
 'me',
 'that',
 'this',
 'means',
 'war,',
 '\n',
 'if',
 'you',
 'still',
 'try',
 'to',
 'defend',
 'the',
 'infamies',
 'and',
 'horrors',
 'perpetrated',
 'by',
 'that',
 '\n',
 'antichrist--i',
 'really',
 'believe',
 'he',
 'is',
 'antichrist--i',
 'will',
 'have',
 'nothing',
 'more',
 '\n',
 'to',
 'do',
 'with',
 'you',
 'and',
 'you',
 'are',
 'no',
 'longer',
 'my',
 'friend,',
 'no',
 'longer',
 'my',
 "'faithful",
 '\n',
 "slave,'",
 'as',
 'you',
 'call',
 'yourself!',
 'but',
 'how',
 'do',
 'you',
 'do?',
 'i',
 'see',
 'i',
 'have',
 'frightened',
 '\n',
 'you--sit',
 'down',
 'and',
 'tell',
 'me',
 'all',
 'the',
 'news."',
 '\n',
 '\n',
 'it',
 'was',
 'in',
 'july,']

In [0]:
# Calculate word frequency
MIN_WORD_FREQUENCY=2
word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

Unique words before ignoring: 39830
Ignoring words with frequency < 2
Unique words after ignoring: 18747


In [0]:
len(word_freq.keys())

39830

In [0]:
words[:100]

['\n',
 '"\'i',
 '"800',
 '"a',
 '"a-tu!"',
 '"about',
 '"adieu,',
 '"afraid',
 '"after',
 '"ah',
 '"ah!',
 '"ah!"',
 '"ah,',
 '"all',
 '"allow',
 '"always',
 '"am',
 '"an',
 '"and',
 '"andrew',
 '"andrew,',
 '"anna',
 '"another',
 '"any',
 '"are',
 '"aren\'t',
 '"arrange',
 '"as',
 '"ask',
 '"at',
 '"bad!',
 '"be',
 '"because',
 '"because,',
 '"before',
 '"believe',
 '"besides',
 '"besides,',
 '"besides,"',
 '"better',
 '"bind',
 '"board',
 '"bolkonski!"',
 '"bonaparte',
 '"bonjour,',
 '"boris',
 '"both',
 '"brigand!',
 '"bring',
 '"brothers!',
 '"brothers!"',
 '"buonaparte',
 '"buonaparte?"',
 '"but',
 '"but,',
 '"but...',
 '"bwing',
 '"by',
 '"c\'est',
 '"call',
 '"calm',
 '"can',
 '"captain',
 '"captain,',
 '"catch',
 '"cette',
 '"charming!"',
 '"christ',
 '"clear',
 '"colonel,"',
 '"come',
 '"come!',
 '"come,',
 '"commander',
 '"confound',
 '"count',
 '"count!',
 '"count,',
 '"countess',
 '"countess,',
 '"cut',
 '"darling',
 '"dear',
 '"dear,',
 '"dear-est!"',
 '"delighted',
 '"de

If we remember, at this point we have text_in_words which is an array containing all the corpus word-by-word. We need to create sequences of size SEQUENCE_LEN (another parameter that can be picked by hand) and store them in sentences, and in the same index, store the next word in next_words.

In [0]:
# cut the text in semi-redundant sequences of SEQUENCE_LEN words
STEP = 1
SEQUENCE_LEN=100
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 585009
Remaining sequences: 38599


In [0]:
sentences, sentences_test, next_words, next_words_test = train_test_split(sentences, next_words,shuffle=True)


In [0]:
len(sentences_test)

9650

In [0]:
model = Sequential()
dropout=3
model.add(Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
if dropout > 0:
    model.add(Dropout(dropout))
model.add(Dense(len(words)))
model.add(Activation('softmax'))


In [0]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 256)               19329024  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 18747)             4817979   
_________________________________________________________________
activation_2 (Activation)    (None, 18747)             0         
Total params: 24,147,003
Trainable params: 24,147,003
Non-trainable params: 0
_________________________________________________________________


Without word filtering, I had roughly 1 million sentences (len(sentences) = 1000000), SEQUENCE_LEN = 10 and 40,000 different words (len(words)=40000). With these numbers, x had a size of 400,000,000,000(!). Considering that in numpy is 1 byte, this gave me an approximate of 400 GB of memory(!).
Hence the need of data generators. Using data generators, you feed the model with chunks of the training set, one for each step, instead of feeding everything at once.

In [0]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y


In [0]:
file_path = "LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}.ckpt" % (
    len(words),
    SEQUENCE_LEN,
    MIN_WORD_FREQUENCY
)
checkpoint_dir=os.path.dirname(file_path)
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback()
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]

In [0]:
'''BATCH_SIZE=200
model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
    epochs=20,
    callbacks=callbacks_list,
    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),              
    validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)'''

Epoch 1/20
  1/145 [..............................] - ETA: 12:22 - loss: 5.7816e-04 - acc: 0.9999

KeyboardInterrupt: ignored

In [0]:
!ls {checkpoint_dir}

drive
LSTM_LYRICS-epoch001
LSTM_LYRICS-epoch001.ckpt
LSTM_LYRICS-epoch001-words18747-sequence100-minfreq2-loss0.0006-acc0.9999-val_loss0.0005-val_acc0.9999.ckpt
LSTM_LYRICS-epoch001-words18747-sequence100-minfreq2-loss0.0006-acc0.9999-val_loss0.0006-val_acc0.9999.ckpt
LSTM_LYRICS-epoch001-words4758-sequence100-minfreq10-loss0.0020-acc0.9998-val_loss0.0020-val_acc0.9998.ckpt
sample_data


In [0]:
model=load_model('LSTM_LYRICS-epoch001-words18747-sequence100-minfreq2-loss0.0006-acc0.9999-val_loss0.0005-val_acc0.9999.ckpt')

In [0]:

"""
Script to generate text from an already trained network (with lstm_train.py)
--By word--
It is necessary to at least provide the trained model and the vocabulary file
(generated also by lstm_train.py).
"""




def validate_seed(words, seed):
    """Validate that all the words in the seed are part of the vocabulary"""
    print("\nValidating that all the words in the seed are part of the vocabulary: ")
    seed_words = seed.split(" ")
    valid = True
    for w in seed_words:
        print(w, end="")
        if w in vocabulary:
            print(" ✓ in vocabulary")
        else:
            print(" ✗ NOT in vocabulary")
            valid = False
    return valid


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def generate_text(model, indices_word, word_indices, seed,
                  sequence_length, quantity):
    """
    Similar to lstm_train::on_epoch_end
    Used to generate text using a trained model
    :param model: the trained Keras model (with model.load)
    :param indices_word: a dictionary pointing to the words
    :param seed: a string to be used as seed (already validated and padded)
    :param sequence_length: how many words are given to the model to generate
    :param diversity: is the "temperature" of the sample function (usually between 0.1 and 2)
    :param quantity: quantity of words to generate
    :return: Nothing, for now only writes the text to console
    """
    #sentence = seed.split(" ")
    #print("----- Generating text")
    #print('----- Diversity:' + str(diversity))
    #print('----- Generating with seed:\n"' + seed)

    sentence=seed.split(" ")
    for i in range(quantity):
        x_pred = np.zeros((1, sequence_length, len(words)))
        for t, word in enumerate(sentence):
            x_pred[0, t, word_indices[word]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_word = indices_word[next_index]

        sentence = sentence[1:]
        sentence.append(next_word)

        print(" "+next_word, end="")
    print("\n")




In [0]:
def seed_generator(sentence,batch_size):
        wo=[]
        batch_size=1
        wo=sentence.split()
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        #y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(wo):
                x[i, t, word_indices[w]] = 1
            #y[i, word_indices[next_word_list[index]]] = 1

            #index = index + 1
            #if index == len(sentence_list):
            #    index = 0
        return x


In [0]:
texttt='a demonstrating reveal found, happen?'

In [0]:
generate_text(model,indices_word,word_indices,texttt,100,100)

 impossibility various either. purse and quit 
 happened?" 
 divided bosom dark think! taken gust knowledge disappeared. solemn, eyebrows, hospital. the giving tastes upset. cases attention. victory pulse, rewards weeping disordered endeavors abandoning preserves cloudlets poetic. progress fox. course! 
 room: move, disagreed gentry blue blame horror krasnoe, battle." scowling, 
 health, 
 kutuzov," questioning (in chairs increased. deceived, millions, people's books, sternly character. conservatory. coming! continued, pond united, 
 dorogobuzh evince short 
 merrily. united, violence. utterance however," undressing, become curls piteous, malevolently. plumes continue. clapping style, bilibin," cold 
 grade, concluded destination. suspicion. frankly scratching 
 trick enormously

